# Customize Enviornment

In [1]:
# set default plot size
options(repr.plot.width=10, repr.plot.height=6)
options(warn = FALSE)

In [3]:
# configure multicore processing
library(doMC)
registerDoMC(cores=8)

In [23]:
library(RMySQL)
library(lubridate)
library(rpart)
library(ggplot2)
library(caret)
library(e1071)
library(DMwR)
library(randomForest)
library(party)
library(Matrix)
library(xgboost)
library(DiagrammeR)
library(pROC)
library(ROSE)
library(ipred)
library(nnet)
library(AnomalyDetection)

ERROR: Error in library(AnomalyDetection): there is no package called ‘AnomalyDetection’


In [26]:
#install.packages("devtools")
#library(devtools)
devtools::install_github("twitter/AnomalyDetection")

from URL https://api.github.com/repos/twitter/AnomalyDetection/zipball/master
Installing AnomalyDetection
Warning message in utils::untar(src, exdir = target, compressed = "gzip"):
“‘/bin/gtar -xf '/tmp/Rtmp1FQ6h8/stringr_1.2.0.tar.gz' -C '/tmp/Rtmp1FQ6h8/devtools62201311d7dd'’ returned error code 127”

ERROR: Error in system(cmd, intern = TRUE): error in running command


# Get Data

In [ ]:
# create an open connection to the database
#cn <- dbConnect(drv = RMySQL::MySQL(),
#                host = "host",
#                port = 3306,
#                user = "user",
#                password = "password",
#                dbname = "db")

# query the database and store the results into a DataFrame
#df <- dbGetQuery(cn, "SELECT * FROM dryer3_fail_1_hour_before")
#dbDisconnect(cn)

In [ ]:
# convert datetime_hour to datetime object
#df$Datetime_hour <- ymd_hms(df$Datetime)

In [9]:
# save df to local machine
#saveRDS(df, "dryer3_fail_1_hour_before.Rda")
df <- readRDS("dryer3_fail_1_hour_before.Rda")

In [10]:
# drop the old failure variable and set the new failure variable as factor
df$Class  <- as.factor(df$NEW_FAIL)
df$FAILURE <- NULL
df$NEW_FAIL <- NULL

# create variable names for the Class variable
levels(df$Class) <- make.names(c("notFail", "fail"))

In [11]:
# create new variables for dates and time
df$Year <- year(df$Datetime_hour)
df$Month <- month(df$Datetime_hour)
df$Day <- day(df$Datetime_hour)
df$Hour <- hour(df$Datetime_hour)
df$Minute <- minute(df$Datetime_hour)
#df$Datetime_hour <- NULL

In [12]:
# look at the number of records per month
table(df$Year, df$Month)

      
         1   2   3   4   5   6   7   8   9  10  11  12
  2015 718 419 733 720 715 704 725 734 648 565 633 531
  2016 450 284 732 717 657 627   0   0   0   0   0   0

In [13]:
# look at the number of failures per month
fails <- df[df$Class == 'fail', ]
table(fails$Year, fails$Month)

      
        1  2  3  4  5  6  7  8  9 10 11 12
  2015 11  4 10  0 10 10  8  4  3  2  2  3
  2016  5  0  3  1  1  0  0  0  0  0  0  0

In [14]:
# subset df to only 2015
df <- df[df$Year == 2015 & df$Month < 8, ]

In [15]:
# look at the number of failures per month
table(df$Class, df$Month)

         
            1   2   3   4   5   6   7
  notFail 707 415 723 720 705 694 717
  fail     11   4  10   0  10  10   8

In [16]:
# look at the number of variables
dim(df)

[1] 4734  114

In [17]:
# drop variables are are constant
df <- df[sapply(df, function(x) length(unique(na.omit(x)))) > 1]

In [18]:
# look at how many variables remain
dim(df)

[1] 4734   83

### Split data into training and test

In [19]:
# split into training and test
#imbal_train  <- df[c(1:5500),]
#imbal_test  <- df[c(5501:7845), ]
imbal_train <- df[df$Month <= 5, ]
imbal_test <- df[df$Month > 5, ]

In [20]:
# proportion of training data
prop.table(table(imbal_train$Class))


   notFail       fail 
0.98940998 0.01059002 

In [21]:
# proportion of test data
prop.table(table(imbal_test$Class))


   notFail       fail 
0.98740378 0.01259622 